In [1]:
from web_scraper.web_scraper import CompendiumScraper
import pandas as pd

# Running To-Do List for Scraper
- Implement logging
- Integrate `screen_for_rework` results into the pipeline (maybe adjust the other main functions so that they have a kwarg to click the GL/JP icon?)
- Add something that will pull character images from their profile and save them in a folder for later use (By.XPATH, "//img[@class='charmanimage']").get_attribute("src")

In [2]:
cs = CompendiumScraper()

In [11]:
cs.generate_ability_dict('amarant')

Unable to access abilities for Amarant.
They might not be released to GL yet.


In [12]:
cs.ability_dict_omnibus

{'reks': {'BRV Attack+': {'ability_attack_info': <selenium.webdriver.remote.webelement.WebElement (session="cf06110a7efa051ce6a9b7c3aaa280f1", element="06411B3CF3F4BDC0A3C44316B30C86E4_element_366")>,
   'attribute_list': ['Melee']},
  'BRV Attack++': {'ability_attack_info': <selenium.webdriver.remote.webelement.WebElement (session="cf06110a7efa051ce6a9b7c3aaa280f1", element="06411B3CF3F4BDC0A3C44316B30C86E4_element_367")>,
   'attribute_list': ['Melee']},
  'HP Attack+': {'ability_attack_info': <selenium.webdriver.remote.webelement.WebElement (session="cf06110a7efa051ce6a9b7c3aaa280f1", element="06411B3CF3F4BDC0A3C44316B30C86E4_element_368")>,
   'attribute_list': ['Melee']},
  'HP Attack++': {'ability_attack_info': <selenium.webdriver.remote.webelement.WebElement (session="cf06110a7efa051ce6a9b7c3aaa280f1", element="06411B3CF3F4BDC0A3C44316B30C86E4_element_369")>,
   'attribute_list': ['Melee']},
  'Blade Combo': {'ability_attack_info': <selenium.webdriver.remote.webelement.WebElemen

In [13]:
cs.parse_ability_dict_to_df('amarant')

No ability_dict for amarant.
Run `generate_ability_dict(amarant)` first, then `parse_ability_dict_to_df()` should work for them.


In [14]:
cs.retrieve_hp_caps_from_bt('amarant', return_output=True)

Unable to find a BT for Amarant. Either the character isn't in GL yet, or they lack a BT in both GL and JP.


UnboundLocalError: cannot access local variable 'bt_button_element' where it is not associated with a value

In [15]:
cs.bt_effect_dict_omnibus

{'reks': {'char_name': 'reks',
  'bt_personal_hp_dmg_cap_up': 0,
  'bt_party_hp_dmg_cap_up': 30}}

In [16]:
cs.retrieve_ha_hp_dmg_cap_up('amarant', return_output=True)

Unable to find High Armor info for Amarant.
They might not be released to GL yet.


In [17]:
cs.ha_dict_omnibus

{'reks': {'char_name': 'reks',
  'personal_hp_dmg_cap_up': 0,
  'party_ha_hp_dmg_cap_up': 15}}

In [19]:
cs.chars_not_in_gl_yet

['amarant', 'amarant']